# Credit Card Transaction Generator using Kakfa

The code that follows is derived from this tutorial:
   https://www.markhneedham.com/blog/2019/05/16/kafka-basic-tutorial/

The execution of this notebook requires a running Zookeeper instance and Kafka broker. On Mac OS, these can be installed with homebrew and launched with the following commands:

```
/usr/local/opt/kafka/bin/zookeeper-server-start /usr/local/etc/kafka/zookeeper.properties

/usr/local/opt/kafka/bin/kafka-server-start /usr/local/etc/kafka/server.properties

pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0
```

note: the Kafka package can be added by adding the following line to pyspark config:

```
File: /usr/local/Cellar/apache-spark/3.5.0/libexec/conf/spark-defaults.conf
...
   spark.jars.packages               org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0
...
```

In [ ]:
# only run this if the kafka-python is not pre-installed in your python environment
# !pip install kafka-python

In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json
import uuid


def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer(server):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[server], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [ ]:
server = 'localhost:9092'
topic_name = "cc_fraud_topic"

In [ ]:
kafka_producer = connect_kafka_producer(server)

In [ ]:
message = "104872,2019-03-02T07:58:56.000-05:00,4430881574719618,fraud_Huel-Langworth,misc_net,1.12,Alicia,Morales,F,4456 Dunn Roads,Meredith,CO,81642,39.3199,-106.6596,61.0,Public relations account executive,1939-11-04,610a5f5782076b7a952824d1d949255d,1330675136,39.635246,-107.054549,0"
publish_message(kafka_producer, topic_name, str(uuid.uuid4()), message)

In [ ]:
message = '94460,2019-02-25T12:44:11.000-05:00,4810839835482794272,"fraud_Weber, Thiel and Hammes",kids_pets,14.83,Carolyn,Perez,F,433 Blake Roads Suite 967,Wheaton,MO,64874,36.7651,-94.0492,760.0,Production manager,1985-08-31,b53b115efdbfe79956452a295f1a8a57,1330173851,36.487874,-94.355554,0'
publish_message(kafka_producer, topic_name, str(uuid.uuid4()), message)

In [ ]:
consumer = KafkaConsumer(topic_name, 
                         auto_offset_reset='earliest',
                         bootstrap_servers=[server], 
                         api_version=(0, 10), 
                         value_deserializer = json.loads,
                         consumer_timeout_ms=1000)
for msg in consumer:
    print(msg.key.decode("utf-8"), msg.value)